# Bash Errors Helper

This notebook provides tools to study why some bash moves knock the player around while others don't. It loads recordings from the `hoplite.controller.Recorder`, look for bash moves, and if so copies the corresponding screenshot either to a `success` subfolder or a `fail` one whether the player position **after** the bash move has been correctly guessed by the game engine. Screenshots are **annotated** with colored circles:
 - Green dots mark the bashed area
 - Blue dot marks the exact bashed tile
 - Orange dot marks the true position after the bash

In [ ]:
# Imports
import os
import glob
import shutil
import matplotlib.image
import matplotlib.pyplot
try:
    import hoplite
except ModuleNotFoundError:
    os.chdir("..")
import hoplite.game.state
import hoplite.game.moves
import hoplite.actuator

In [ ]:
# Main script configuration
OUTPUT_DIRECTORY = "draft/bash"
RECORDINGS_FOLDER = "recordings"
LOG_FILENAME = "game.log"

## 1. Screenshot Annotation

Function drawing the appropriate circles on a given screenshots.

In [ ]:
class Colors:
    
    GREEN = [0., 1., 0., 1.]
    BLUE = [.2, .6, 1., 1.]
    ORANGE = [1, .6, .1, 1.]


def circle(array, position, color, diameter=50, corner=500):
    coordinates = hoplite.actuator.hexagonal_to_pixels(position)
    radius = diameter // 2
    for x in range(diameter):
        for y in range(diameter):
            if (x - radius) ** 2 + (y - radius) ** 2 >= corner:
                continue
            array[
                coordinates[1] + y - radius,
                coordinates[0] + x - radius
            ] = color[:]


def transform(prev_state, next_state, move, path):
    bashed_area = move._get_bashed_area(prev_state)
    screenshot = matplotlib.image.imread(path)
    size = 50
    for position in bashed_area:
        circle(screenshot, position, Colors.GREEN)
    circle(screenshot, move.target, Colors.BLUE)
    circle(
        screenshot,
        next_state.terrain.player,
        Colors.ORANGE,
        corner=250
    )
    return screenshot

matplotlib.pyplot.figure(figsize=(6, 10.67))
prev_state = hoplite.game.state.GameState.from_string(
    "15;0000010300500100021120110090103000000a02006b0001902500000100c000000000000003000;0/135/1/6/2/7,4,6,11,5,13,9,3,10,1,1,1,1,1")
next_state = hoplite.game.state.GameState.from_string(
    "15;00500100000001000211001100a0123600000020500b0001020000000100c000000000000000000;0/135/1/4/0/7,4,6,11,5,13,9,3,10,1,1,1,1,1")
move = hoplite.game.moves.PlayerMove.from_string("bash/-1,-2")
matplotlib.pyplot.imshow(transform(
    prev_state,
    next_state,
    move,
    "recordings/005/263.png"
))

## 2. Log Parsing

Actual game recordings checking.

In [ ]:
def ifn(folder, turn):
    return os.path.join(
        folder, 
        str(turn).rjust(3, "0") + ".png"
    )


def ofn(folder, turn, error):
    return os.path.join(
        OUTPUT_DIRECTORY,
        {
            True: "fail",
            False: "success"
        }[error],
        "%s-%s.png" % (
            os.path.basename(folder),
            str(turn).rjust(3, "0")
        )
    )


def parse(prev_line, next_line):
    none = None, None, None
    if prev_line.split("\t")[1] != "move":
        return none
    if next_line.split("\t")[1] != "move":
        return none
    prev_state = hoplite.game.state.GameState.from_string(prev_line.split("\t")[2])
    next_state = hoplite.game.state.GameState.from_string(next_line.split("\t")[2])
    if prev_state.depth != next_state.depth:
        return none
    move = hoplite.game.moves.PlayerMove.from_string(
        prev_line.split("\t")[3])
    if not move.__class__ == hoplite.game.moves.BashMove:
        return none
    return prev_state, next_state, move


def predict(prev_state, move):
    predicted = move.apply(prev_state)
    predicted.status.cooldown = max(
        0, 
        predicted.status.cooldown - 1
    )
    return predicted


for suffix in ["fail", "success"]:
    os.makedirs(
        os.path.join(OUTPUT_DIRECTORY, suffix),
        exist_ok=True
    )

for folder in glob.glob(os.path.join(RECORDINGS_FOLDER, "*")):
    filename = os.path.join(folder, LOG_FILENAME)
    with open(filename) as file:
        lines = file.readlines()
    print("Read %d lines from %s" % (len(lines), filename))
    for prev_line, next_line in zip(lines[:-1], lines[1:]):
        prev_state, next_state, move = parse(prev_line, next_line)
        if prev_state is None:
            continue
        turn = int(prev_line.split("\t")[0])
        predicted = predict(prev_state, move)
        screenshot = transform(
            prev_state,
            next_state,
            move,
            ifn(folder, turn)
        )
        error = predicted.terrain.player != next_state.terrain.player
        print("Bash move at turn %d: %s" % (turn, {
            True: "fail",
            False: "success"
        }[error]))
        matplotlib.image.imsave(
            ofn(folder, turn, error),
            screenshot
        )